## Dependencies

In [1]:
# System dependencies
from os import listdir
import time as clock
from datetime import timedelta
from IPython.display import clear_output
import pickle

In [2]:
# Data manipulation dependencies
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
# Custom data handling methods
import prep_IO

In [4]:
# Directory of pre-determined lustre failed jobs
#1) emailed set, 2) emailed set
#first_src = './lustre_fail_set.txt'

# 3) 9/21 - 9/23 4) 10/14
#rem_src = [ '/oasis/projects/nsf/sys200/stats/xsede_stats/comet_accounting/2019-09-21.txt',
#        '/oasis/projects/nsf/sys200/stats/xsede_stats/comet_accounting/2019-09-22.txt',
#        '/oasis/projects/nsf/sys200/stats/xsede_stats/comet_accounting/2019-09-23.txt',
#        '/oasis/projects/nsf/sys200/stats/xsede_stats/comet_accounting/2019-10-14.txt']
src =  './lustre_set.pkl'

## Data

In [5]:
# Given: 'JobID|User|Account|Start|End|Submit|Partition|Timelimit|JobName|State|NNodes|ReqCPUS|NodeList\n'
# DNI: User, Account, JobName
# Else: OK

In [8]:
# temp test
f = open(src,"rb")
test_out = pickle.load( f )
len(test_out)

113624

In [9]:
test_out[:10]

[('comet-14-72', '2020-05-13T02:11:38', '2020-05-13T05:07:34', '33301074'),
 ('comet-19-19', '2020-05-13T02:11:38', '2020-05-13T05:07:34', '33301074'),
 ('comet-28-55', '2020-05-13T02:11:38', '2020-05-13T05:07:34', '33301074'),
 ('comet-30-10', '2020-05-13T10:50:08', '2020-05-13T10:50:12', '33321014'),
 ('comet-06-46', '2020-05-12T06:40:43', '2020-05-13T14:27:44', '33283100'),
 ('comet-12-52', '2020-05-12T06:40:43', '2020-05-13T14:27:44', '33283100'),
 ('comet-22-39', '2020-05-12T06:40:43', '2020-05-13T14:27:44', '33283100'),
 ('comet-22-64', '2020-05-12T06:40:43', '2020-05-13T14:27:44', '33283100'),
 ('comet-21-07', '2020-05-27T08:49:26', '2020-05-28T01:27:49', '33637231'),
 ('comet-22-48', '2020-05-27T09:32:35', '2020-05-28T00:25:06', '33637422')]

In [ ]:
search_out_A = prep_IO.search( mode='l', from_list=test_out )

save_thing = search_out_A
f = open("lustre_set_files_A.pkl","wb")
pickle.dump( save_thing,f )
f.close()

len(search_out_A)

In [ ]:
search_out_B = prep_IO.deep_search_host( test_out, jobids=True )

save_thing = search_out_B
f = open("lustre_set_files_B.pkl","wb")
pickle.dump( save_thing,f )
f.close()

len(search_out_B)

#### PAUSE FOR DEBUG

In [ ]:
def fill_host_source( deep_out_dict ):
    return { search_key : host_to_info_dict( source_dict["Source"][0] )
                for search_key,source_dict in deep_out_dict.items() }

def host_to_info_dict( zip_txt, find_id=False ):
    l = zip_txt.find("comet")
    r = zip_txt.rfind("comet")
    
    if l != r:
        return zip_txt
    
    contents = prep_IO.unzip_txt( zip_txt )
    #host_name = contents[1].partition(" ")[2][:11]
    info_dict = { "Schemas":{}, "Specs":[],
                  "Data":{}, "Jobid(s)":[]
                }
    data_start = 0
    
    for i in range(len( contents )):
        line = contents[ i ]
        
        # Spec line
        if line[0] == "$":
            info_dict["Specs"].append( prep_IO.format_spec( line ) )
        
        # Schema line    
        elif line[0] == "!":
            info_dict["Schemas"].update( prep_IO.format_schema( line ) )
                
        # Start of variable content options
        else:
            data_start = i
    
    curr_header = ''
    for j in range( data_start, len(contents) ):
        line = contents[ j ]

        # Header line
        if prep_IO.check_header( line ):
            header_dict = prep_IO.format_header( line )
            curr_header = line
                                        
            if header_dict:
                info_dict["Data"][ curr_header ] = {} 
                    
            if prep_IO.check_job( header_dict["Jobid"] ):
                curr_jobid = header_dict["Jobid"]
                info_dict["Jobid(s)"].append( curr_jobid )

        # Data line
        else:
            curr_data = prep_IO.format_data( line )
            info_dict["Data"][curr_header].update( curr_data )
    
    return info_dict

In [ ]:
deep_out = deep_search_host( test_out, jobids=True )
deep_out

In [ ]:
test_key = ('comet-14-72',
  '2020-05-13T02:11:38',
  '2020-05-13T05:07:34',
  '33301074')

host_to_info_dict( deep_out[test_key]["Source"][0] )

#### RESUME

In [ ]:
keys = list(search_out.keys())
len(keys)

In [ ]:
sample_key = keys[0]
sample = search_out[ sample_key ]
print( sample_key )
print( sample.keys() )

In [ ]:
sample["Source Files"]

In [ ]:
sample['Acct Info']

In [ ]:
sample['Host Info'].keys()

In [ ]:
sample['Host Info']["Specs"]

In [ ]:
sample['Host Info']["Timely Data"]

# Save search_out for demo_ use

In [ ]:
save_thing = deep_out
f = open("lustre_set_files.pkl","wb")
pickle.dump( save_thing,f )
f.close()